In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

# We will be solving the burger's equation using the Rancic method. 

In [46]:
# Parameters
T = 100
dt = 1

In [60]:
x_coordinates = np.array([x for x in range(-50, 51)])
initial_distribution = 0.5*np.sin(np.pi*x_coordinates/50)

grid = np.zeros((T, len(x_coordinates)))
grid[0, :] = initial_distribution

In [54]:
# finite difference differentiator 
def finite_diff(row):
    diff_mat = (np.roll(row, -1) - np.roll(row, 1))/2
    return diff_mat

def two_step_vel(neg_1_row, neg_2_row):
    vel_row = (3/2)*neg_1_row - (1/2)*neg_2_row
    return vel_row

def three_step_vel(neg_1_row, neg_2_row, neg_3_row):
    vel_row = (15/8)*neg_1_row - (10/8)*neg_2_row + (3/8)*neg_3_row
    return vel_row

def alpha_iter(n_step_vel, tolerance = 10**(-6), max_iter = 10000):
    alpha = np.zeros(len(n_step_vel))
    alpha_0 = 0
    for x in range(len(n_step_vel)):
        alpha_prev = alpha_0
        for i in range(max_iter):
            alpha_new = (1 + np.floor(x - alpha_prev) - (x - alpha_prev))*n_step_vel[np.floor(x - alpha_prev).astype(int)%len(n_step_vel)] + ((x - alpha_prev) - np.floor(x - alpha_prev))*n_step_vel[np.ceil(x - alpha_prev).astype(int)%len(n_step_vel)]
            if np.abs(alpha_new - alpha_prev) < tolerance:
                alpha[x] = alpha_new
                break
            alpha_prev = alpha_new
    return alpha



In [23]:
alpha = alpha_iter(np.ones(len(x_coordinates)))
print(alpha)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1.]


In [61]:
for t in range(1, T):
    if t == 1:
        alpha = alpha_iter(grid[t-1, :])
#        alpha = alpha_iter(1.23*np.ones(len(x_coordinates)))

        p = np.round(alpha).astype(int)
        t_eulerian_timestep = np.where(alpha != 0, dt*(1 - p/alpha), 0)
    elif t == 2:
        alpha = alpha_iter(two_step_vel(grid[t-1, :], grid[t-2, :]))
#        alpha = alpha_iter(two_step_vel(1.23*np.ones(len(x_coordinates)), 1.23*np.ones(len(x_coordinates))))

        p = np.round(alpha).astype(int)
        t_eulerian_timestep = np.where(alpha != 0, dt*(1 - p/alpha), 0)
    else:
        alpha = alpha_iter(three_step_vel(grid[t-1, :], grid[t-2, :], grid[t-3, :]))
#        alpha = alpha_iter(three_step_vel(1.23*np.ones(len(x_coordinates)), 1.23*np.ones(len(x_coordinates)), 1.23*np.ones(len(x_coordinates))))

        p = np.round(alpha).astype(int)
        t_eulerian_timestep = np.where(alpha != 0, dt*(1 - p/alpha), 0)

    diff_grid = finite_diff(grid[t-1, :])
    double_diff_grid = finite_diff(diff_grid)
    for x in range(len(x_coordinates)):
        grid[t, x] = grid[t-1, (x - p[x])%len(x_coordinates)] - t_eulerian_timestep[(x - p[x])%len(x_coordinates)]*diff_grid[(x - p[x])%len(x_coordinates)] + 0.5*t_eulerian_timestep[(x - p[x])%len(x_coordinates)]**2*double_diff_grid[(x - p[x])%len(x_coordinates)]

/var/folders/6q/_skb6gvs4b51jr_z4v90tvt40000gn/T/ipykernel_43029/1618677872.py:7: RuntimeWarning: invalid value encountered in divide
  t_eulerian_timestep = np.where(alpha != 0, dt*(1 - p/alpha), 0)
/var/folders/6q/_skb6gvs4b51jr_z4v90tvt40000gn/T/ipykernel_43029/1618677872.py:13: RuntimeWarning: invalid value encountered in divide
  t_eulerian_timestep = np.where(alpha != 0, dt*(1 - p/alpha), 0)
/var/folders/6q/_skb6gvs4b51jr_z4v90tvt40000gn/T/ipykernel_43029/1618677872.py:19: RuntimeWarning: invalid value encountered in divide
  t_eulerian_timestep = np.where(alpha != 0, dt*(1 - p/alpha), 0)


In [59]:
from video_creator_1d import create_video
create_video(grid, "check.mp4")